In [1]:
%matplotlib inline
import xarray as xr
import os
import numpy as np
import matplotlib.pyplot as plt
import collections
import warnings 
from netCDF4 import default_fillvals
from scipy.stats import hmean

# import veg functions
from parameter_functions import (calculate_cv_pft, map_pft_to_nldas_class, is_overstory, 
                                 calc_root_fract, calc_root_depth_rz1, calc_root_depth_rz2)

from parameter_functions_v2 import (create_empty_arrays, 
                                    create_parameter_dataset, calculate_nveg_pfts)

from veg_parameter_functions import create_veg_parameter_dataset

# define fillvals
fillval_f = default_fillvals['f8']
fillval_i = default_fillvals['i4']

This is a notebook for deriving VIC 5 vegetation parameters for the NCAR RAL group, to be used for high-resolution simulations over Alaska. 

__Set resolution for filenames__

In [2]:
res = '12km'

# set file extensions 
if res == "50km":
    grid = 'wr50a_ar9v4'
elif res == "25km":
    grid = 'wr25b_ar9v4'
elif res == "12km":
    grid = 'ncar_ral'
    
if res == "50km":
    nj = 205
    ni = 275
elif res == "25km":
    nj = 413
    ni = 551
elif res == "12km":
    nj = 209
    ni = 299
    
num_veg = 17
print("calculating parameters at %s" %res)

calculating parameters at 12km


__Set domain file__

In [3]:
if res == '50km':
    domain = xr.open_dataset(os.path.join('/u/home/gergel/data/parameters', 
                                      'domain.lnd.wr50a_ar9v4.100920.nc'))
elif res == "25km":
    domain = xr.open_dataset(os.path.join('/u/home/gergel/data/parameters', 
                                      'domain.lnd.wr25b_ar9v4.170413.nc'))
elif res == "12km":
    domain = xr.open_dataset(os.path.join('/u/home/gergel/data/parameters', 
                                      'alaska_vic_domain_ncar.nc'))
masknan_vals = domain['mask'].where(domain['mask'] == 1).values

In [4]:
old_params = xr.open_dataset(os.path.join('/u/home/gergel/data', 
                                          'vic_params_wr50a_vic5.0.dev_20160328.nc'))

__Options__

In [5]:
# if set to True, add additional organic_fract options to parameter file, including soil organic fraction, 
# soil particle density of OM, and bulk density of OM 
organic_fract = True
max_snow_albedo = True
bulk_density_comb = True

load CLM PFTs to use for vegetation parameters

In [6]:
# load regridded PFTs 
pfts_data_dir = '/u/home/gergel/data/parameters/pfts/regridded_pfts'
pfts_filename = 'mksrf_landuse_rc2000_c110913_ncar.nc'
veg_data = xr.open_dataset(os.path.join(pfts_data_dir, pfts_filename))

__Calculate Cv from PFTs__

In [7]:
cv = xr.apply_ufunc(calculate_cv_pft, 
                    veg_data['PCT_PFT'].where(domain.mask == 1),
                    dask='allowed',
                    vectorize=True)

__Calculate number of active PFTs, `Nveg`__ 

In [8]:
Nveg = xr.apply_ufunc(calculate_nveg_pfts,
                      veg_data['PCT_PFT'].where(domain.mask == 1),
                      dask='allowed',
                      input_core_dims=[['pft']],
                      vectorize=True)

__Load LAI and vegetation height, `MONTHLY_LAI` and `MONTHLY_HEIGHT_TOP`__

In [9]:
lai_file = xr.open_dataset(os.path.join('/u/home/gergel/data/parameters/lai/regridded_lai', 
                                   'mksrf_lai_78pfts_simyr2005.c170413_ncar_lai.nc'))
veg_height_file = xr.open_dataset(os.path.join('/u/home/gergel/data/parameters/lai/regridded_lai', 
                                'mksrf_lai_78pfts_simyr2005.c170413_ncar_veg_height.nc'))

LAI and veg_height from CLM and `PCT_PFT` from CLM have a different number of PFTs (`PCT_PFT` has one more PFT, 17 vs 16). The extra PFT in `PCT_PFT` has `PCT_PFT` = 0 over the entire RASM domain, so I just slice the LAI and veg_height from the 0th PFT (water/bare soil) and concatenate it for the 16th PFT. 

In [10]:
lai_slice = lai_file['MONTHLY_LAI'].isel(pft = 0)
vegheight_slice = veg_height_file['MONTHLY_HEIGHT_TOP'].isel(pft=0)

In [11]:
lai = xr.concat([lai_file['MONTHLY_LAI'], lai_slice], dim='pft')
veg_height = xr.concat([veg_height_file['MONTHLY_HEIGHT_TOP'], vegheight_slice], dim='pft')

`veg_rough` = 0.123 * `veg_height`

`displacement` = 0.67 * `veg_height`

In [12]:
veg_rough = 0.123 * veg_height
displacement = 0.67 * veg_height

displacement.values[displacement.values == 0] = 1.0

__Change dims and order of dims of LAI array__

In [13]:
lai = lai.rename({'time': 'month', 'pft': 'veg_class'})
lai = lai.transpose('veg_class', 'month', 'nj', 'ni')

veg_rough = veg_rough.rename({'time': 'month', 'pft': 'veg_class'})
veg_rough = veg_rough.transpose('veg_class', 'month', 'nj', 'ni')

displacement = displacement.rename({'time': 'month', 'pft': 'veg_class'})
displacement = displacement.transpose('veg_class', 'month', 'nj', 'ni')

__Note__: map albedo, root zone fraction and root zone depth based on vegetation type. see `deriving_new_parameters_v2.xlsx` sheet titled `PFT-NLDAS Mapping` for mapping between NLDAS vegetation classes (used in old VIC 5 parameters) and CLM PFTs. This mapping is based on obvious relationships and some approximations (used for PFTs 8-11).

__Create Dataset for variables and define data_vars__

In [14]:
arr_months, arr_nlayer, \
arr_rootzone, arr_veg_classes, \
arr_veg_classes_rootzone, arr_veg_classes_month = create_empty_arrays(domain, nj, ni, num_veg)

In [15]:
# create DataSet
params = create_veg_parameter_dataset(domain, old_params, nj, ni, num_veg, max_snow_albedo)

# fill in values
params['Cv'].values = cv.values
params['Nveg'].values = Nveg.values
params['LAI'].values = lai.values.reshape(num_veg, 12, nj, ni)
params['displacement'].values = displacement.values.reshape(num_veg, 12, nj, ni)
params['veg_rough'].values = veg_rough.values.reshape(num_veg, 12, nj, ni)

define `trunk_ratio`, `rarc`, `rmin`, `wind_h`, `RGL`, `rad_atten`, `wind_atten`, `overstory`, `max_snow_albedo`

In [16]:
# trunk ratio, rarc, rad_atten
trunk_ratio = np.copy(arr_veg_classes)
params['trunk_ratio'].values = trunk_ratio * 0.2
# adjust for bare soil 
params['trunk_ratio'].values[0, :, :] = 0.0

rarc = np.copy(arr_veg_classes)
params['rarc'].values = rarc * 60
# adjust for bare soil
params['rarc'].values[0, :, :] = 100

rad_atten = np.copy(arr_veg_classes)
params['rad_atten'].values = rad_atten * 0.5
# adjust for bare soil 
params['rad_atten'].values[0, :, :] = 0.0

wind_atten = np.copy(arr_veg_classes)
params['wind_atten'].values = wind_atten * 0.5
# adjust for bare soil 
params['wind_atten'].values[0, :, :] = 0.0

In [17]:
if max_snow_albedo == True:
    # max_albedo
    for pft in veg_data.pft.values:
        # get nldas mapping from pft
        nldas = map_pft_to_nldas_class(pft)
        if nldas == 0:
            max_alb = 0.34
        elif nldas == 1:
            max_alb = 0.37
        elif nldas == 2:
            max_alb = 0.35
        elif nldas == 3: 
            max_alb = 0.35
        elif nldas == 4: 
            max_alb = 0.44
        elif nldas == 5:
            max_alb = 0.69
        elif nldas == 6:
            max_alb = 0.43
        elif nldas == 7:
            max_alb = 0.56
        elif nldas == 8:
            max_alb = 0.70
        elif nldas == 9:
            max_alb = 0.65
        elif nldas == 10:
            max_alb = 0.46
        elif nldas == 11:
            max_alb = 0.84
        params['max_snow_albedo'].values[pft, :, :] = np.ones((1, nj, ni)) * max_alb

In [18]:
# rmin, wind_h
for pft in veg_data.pft.values:
    # get nldas mapping from pft
    nldas = map_pft_to_nldas_class(pft)
    if nldas >= 0 and nldas <= 3:
        rmin = np.asscalar(old_params.rmin.isel(veg_class=0).mean())
        wind_h = np.asscalar(old_params.wind_h.isel(veg_class=0).mean())
    elif nldas == 4:
        rmin = np.asscalar(old_params.rmin.isel(veg_class=4).mean())
        wind_h = np.asscalar(old_params.wind_h.isel(veg_class=4).mean())
    elif nldas >= 5 and nldas <= 6:
        rmin = np.asscalar(old_params.rmin.isel(veg_class=5).mean())
        wind_h = np.asscalar(old_params.wind_h.isel(veg_class=5).mean())
    elif nldas >= 7 and nldas <= 8:
        rmin = np.asscalar(old_params.rmin.isel(veg_class=7).mean())
        wind_h = np.asscalar(old_params.wind_h.isel(veg_class=7).mean())
    elif nldas == 9:
        rmin = np.asscalar(old_params.rmin.isel(veg_class=9).mean())
        wind_h = np.asscalar(old_params.wind_h.isel(veg_class=9).mean())
    elif nldas == 10:
        rmin = np.asscalar(old_params.rmin.isel(veg_class=10).mean())
        wind_h = np.asscalar(old_params.wind_h.isel(veg_class=10).mean())
    elif nldas == 11:
        rmin = np.asscalar(old_params.rmin.isel(veg_class=11).mean())
        wind_h = np.asscalar(old_params.wind_h.isel(veg_class=11).mean())
    params['rmin'].values[pft, :, :] = np.ones((1, nj, ni)) * rmin
    params['wind_h'].values[pft, :, :] = np.ones((1, nj, ni)) * wind_h

In [19]:
# RGL
for pft in veg_data.pft.values:
    # get nldas mapping from pft
    nldas = map_pft_to_nldas_class(pft)
    if nldas >= 0 and nldas <= 3:
        rgl = np.asscalar(old_params.wind_h.isel(veg_class=0).mean())
    elif nldas >= 4 and nldas <= 5:
        rgl = np.asscalar(old_params.wind_h.isel(veg_class=4).mean())
    elif nldas >= 6 and nldas <= 8:
        rgl = np.asscalar(old_params.wind_h.isel(veg_class=6).mean())
    elif nldas >= 9 and nldas <= 10:
        rgl = np.asscalar(old_params.wind_h.isel(veg_class=9).mean())
    elif nldas == 11:
        rgl = np.asscalar(old_params.wind_h.isel(veg_class=11).mean())
    params['RGL'].values[pft, :, :] = np.ones((1, nj, ni)) * rgl

In [20]:
# overstory
overstory = np.copy(arr_veg_classes)
for pft in veg_data.pft.values:
    nldas = map_pft_to_nldas_class(pft)
    if nldas > 6:
        # no overstory
        overstory = 0.0
    else: 
        overstory = 1.0
    params['overstory'].values[pft, :, :] = overstory

root fraction and root depth 

In [21]:
root_depth_rz1 = xr.apply_ufunc(calc_root_depth_rz1,
                           params['Cv'].where(domain.mask == 1), 
                           dask='allowed',
                           vectorize=True)
root_depth_rz2 = xr.apply_ufunc(calc_root_depth_rz2,
                           params['Cv'].where(domain.mask == 1), 
                           dask='allowed',
                           vectorize=True)
root_depth = xr.concat([root_depth_rz1, root_depth_rz2],
                      dim='root_zone').transpose('veg_class', 'root_zone', 'nj', 'ni')

In [22]:
# root fract 

rz = 0
for pft in veg_data.pft.values:
    if pft == 0:
        root_fract_rz1 = xr.apply_ufunc(calc_root_fract,
                                        params['Cv'].isel(veg_class=pft),
                                        str(pft),
                                        str(rz),
                                        dask='allowed',
                                        vectorize=True)
    else: 
        root_fract_rz1 = xr.concat([root_fract_rz1, xr.apply_ufunc(calc_root_fract,
                                                                   params['Cv'].isel(veg_class=pft),
                                                                   str(pft),
                                                                   str(rz),
                                                                   dask='allowed',
                                                                   vectorize=True)],
                                  dim='veg_class')
rz = 1
for pft in veg_data.pft.values:
    if pft == 0:
        root_fract_rz2 = xr.apply_ufunc(calc_root_fract,
                                        params['Cv'].isel(veg_class=pft), 
                                        str(pft),
                                        str(rz),
                                        dask='allowed',
                                        vectorize=True)
    else: 
        root_fract_rz2 = xr.concat([root_fract_rz2, xr.apply_ufunc(calc_root_fract,
                                                                   params['Cv'].isel(veg_class=pft), 
                                                                   str(pft),
                                                                   str(rz),
                                                                   dask='allowed',
                                                                   vectorize=True)],
                                  dim='veg_class')
        
root_fract = xr.concat([root_fract_rz1, root_fract_rz2], dim='root_zone').transpose('veg_class', 'root_zone', 'nj', 'ni')

In [23]:
params['root_depth'].values = root_depth
params['root_fract'].values = root_fract

__albedo__

In [24]:
# loop over pft classes and months 
for pft in veg_data.pft.values:
    for month in old_params.month.values:
        nldas = map_pft_to_nldas_class(pft)
        if nldas == 0 or nldas == 1:
            albedo = 0.12
        elif nldas >= 2 and nldas <= 5:
            albedo = 0.18
        elif nldas >= 6 and nldas <= 8:
            albedo = 0.19
        elif nldas == 9:
            albedo = 0.2
        elif nldas == 10: 
            albedo = 0.12
        elif nldas == 11: 
            albedo = 0.2
        params['albedo'].values[pft, month-1, :, :] = np.ones((1, 1, nj, ni)) * albedo

In [25]:
if max_snow_albedo == True:
    veg_class_vars = ['Cv', 'trunk_ratio', 'rarc', 'rmin', 'wind_h', 'RGL', 'rad_atten',
                  'wind_atten', 'albedo', 'LAI', 'overstory',
                  'root_depth', 'root_fract', 'displacement', 'veg_rough', 'max_snow_albedo']
else: 
    veg_class_vars = ['Cv', 'trunk_ratio', 'rarc', 'rmin', 'wind_h', 'RGL', 'rad_atten',
                  'wind_atten', 'albedo', 'LAI', 'overstory',
                  'root_depth', 'root_fract', 'displacement', 'veg_rough']

for veg_class_var in veg_class_vars: 
    if ((veg_class_var == "LAI") or (veg_class_var == "albedo") or (veg_class_var == "root_depth") 
    or (veg_class_var == "root_fract") or (veg_class_var == "veg_rough") or 
        (veg_class_var == "displacement")):
        bare = np.copy(params[veg_class_var].isel(veg_class=0))
        last = np.copy(params[veg_class_var].isel(veg_class=16))
        params[veg_class_var].values[0, :, :, :] = last
        params[veg_class_var].values[16, :, :, :] = bare
    else:
        bare = np.copy(params[veg_class_var].isel(veg_class=0))
        last = np.copy(params[veg_class_var].isel(veg_class=16))
        params[veg_class_var].values[0, :, :] = last
        params[veg_class_var].values[16, :, :] = bare
params['root_fract'].values[16, :, :, :] = 0
params['root_depth'].values[16, :, :, :] = 0
params['displacement'].values[16, :, :, :] = 0
params['veg_rough'].values[16, :, :, :] = 0
params['overstory'].values[16, :, :] = 0

In [26]:
# adjust data vars that need adjusting 
params['Nveg'].values = params['Nveg'].where(domain.mask == 1)
params['trunk_ratio'] = params['trunk_ratio'].where(domain.mask == 1)
params['rarc'] = params['rarc'].where(domain.mask == 1)
params['rmin'] = params['rmin'].where(domain.mask == 1)
params['wind_h'] = params['wind_h'].where(domain.mask == 1)
params['RGL'] = params['RGL'].where(domain.mask == 1)
params['rad_atten'] = params['rad_atten'].where(domain.mask == 1)
params['wind_atten'] = params['wind_atten'].where(domain.mask == 1)
params['root_depth'] = params['root_depth'].where(domain.mask == 1)
params['root_fract'] = params['root_fract'].where(domain.mask == 1)
params['albedo'] = params['albedo'].where(domain.mask == 1)
params['LAI'] = params['LAI'].where(domain.mask == 1)
params['overstory'] = params['overstory'].where(domain.mask == 1)
params['displacement'] = params['displacement'].where(domain.mask == 1)
params['veg_rough'] = params['veg_rough'].where(domain.mask == 1)
if max_snow_albedo == True:
    params['max_snow_albedo'] = params['max_snow_albedo'].where(domain.mask == 1)

In [27]:
encoding_params = {'Nveg': {'dtype': 'int32', "_FillValue": fillval_i},
                   'overstory': {'dtype': 'int32', "_FillValue": fillval_i}}

direc = '/u/home/gergel/data/parameters'
if organic_fract == True and bulk_density_comb == True and max_snow_albedo == True:
    filename = 'new_vic5_params_%s_%s.nc' %(grid, "all_options")
elif organic_fract == False and bulk_density_comb == True and max_snow_albedo == False:
    filename = 'new_vic5_params_%s_%s.nc' %(grid, "bulk_density")
else: 
    filename = 'new_vic5_params_%s_%s.nc' %(grid, "no_options")
new_params_file = os.path.join(direc, filename)
params.to_netcdf(new_params_file, format='NETCDF4_CLASSIC', encoding=encoding_params)

if organic_fract:
    print("Organic Fract is True, including soil_dens_org and organic_fract in parameters file")
if max_snow_albedo:
    print("Max Snow Albedo is True, including veg-dep snow albedo in parameters file")
if bulk_density_comb:
    print("Bulk Density Comb is True, including combined bulk density in parameters file")
    
print("saved new parameters to %s" %new_params_file)

Organic Fract is True, including soil_dens_org and organic_fract in parameters file
Max Snow Albedo is True, including veg-dep snow albedo in parameters file
Bulk Density Comb is True, including combined bulk density in parameters file
saved new parameters to /u/home/gergel/data/parameters/new_vic5_params_ncar_ral_all_options.nc
